In [1]:
!pip install transformers datasets librosa torchaudio torch sklearn matplotlib seaborn

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Define the path to the RAVDESS dataset
RAVDESS_DIR = "/Ravdess/Actor_01"

# Emotion mapping for RAVDESS
emotion_map = {
    1: "neutral", 2: "calm", 3: "happy", 4: "sad",
    5: "angry", 6: "fearful", 7: "disgust", 8: "surprised"
}

# Load dataset into a DataFrame
def load_ravdess_dataset(directory):
    data = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                path = os.path.join(root, file)
                # Extract emotion label from filename
                try:
                    emotion_code = int(file.split("-")[2])
                    emotion = emotion_map.get(emotion_code, "unknown")
                    data.append({"path": path, "emotion": emotion})
                except (IndexError, ValueError):
                    print(f"Skipping file due to filename format issue: {file}")
    return pd.DataFrame(data)

# Load dataset
df = load_ravdess_dataset(RAVDESS_DIR)

# Ensure dataset is not empty
if df.empty:
    raise ValueError("Dataset is empty. Check the dataset path and file structure.")

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["emotion"])

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Print dataset summaries
print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))


ModuleNotFoundError: No module named 'datasets'

In [5]:
pip install datasets


  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-win_amd64.whl.metadata (8.0 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp310-cp310-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp310-cp310-win_amd64.whl.metadata (71 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Define the path to the RAVDESS dataset
RAVDESS_DIR = "Ravdess/Actor_01"

# Emotion mapping for RAVDESS
emotion_map = {
    1: "neutral", 2: "calm", 3: "happy", 4: "sad",
    5: "angry", 6: "fearful", 7: "disgust", 8: "surprised"
}

# Load dataset into a DataFrame
def load_ravdess_dataset(directory):
    data = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                path = os.path.join(root, file)
                # Extract emotion label from filename
                try:
                    emotion_code = int(file.split("-")[2])
                    emotion = emotion_map.get(emotion_code, "unknown")
                    data.append({"path": path, "emotion": emotion})
                except (IndexError, ValueError):
                    print(f"Skipping file due to filename format issue: {file}")
    return pd.DataFrame(data)

# Load dataset
df = load_ravdess_dataset(RAVDESS_DIR)

# Ensure dataset is not empty
if df.empty:
    raise ValueError("Dataset is empty. Check the dataset path and file structure.")

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["emotion"])

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Print dataset summaries
print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))


Train dataset size: 48
Test dataset size: 12


In [8]:
from transformers import Wav2Vec2FeatureExtractor

# Load feature extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

# Preprocess function
def preprocess_function(examples):
    audio_arrays = [librosa.load(path, sr=16000)[0] for path in examples["path"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=16000, padding=True, return_tensors="pt"
    )
    return inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

C:\Users\SASAPU TARUN\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SASAPU TARUN\.cache\huggingface\hub\models--facebook--wav2vec2-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [11]:
from sklearn.preprocessing import LabelEncoder

# Fit the encoder on the entire dataset
label_encoder = LabelEncoder()
label_encoder.fit(df["emotion"])  # Fit on all unique labels

# Apply encoding to train and test datasets
train_dataset = train_dataset.map(lambda x: {"label": label_encoder.transform([x["emotion"]])[0]})
test_dataset = test_dataset.map(lambda x: {"label": label_encoder.transform([x["emotion"]])[0]})


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [12]:
from transformers import Wav2Vec2ForSequenceClassification

# Load model
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=len(emotion_map)
)

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

C:\Users\SASAPU TARUN\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install -U "transformers[torch]" "accelerate>=0.26.0"



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
pip install -U accelerate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
!pip install --upgrade accelerate



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import accelerate
print(accelerate.__version__)


1.3.0


In [4]:
import numpy as np
import torch
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Ensure you have a defined model, train_dataset, and test_dataset
# model = ...
# train_dataset = ...
# test_dataset = ...

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none"  # Disable reporting to external trackers like WandB
)

# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


NameError: name 'train_dataset' is not defined

In [40]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import torch
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score

# Define model and tokenizer
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("glue", "mrpc")

# Prepare datasets
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none"  # Disable reporting to external trackers like WandB
)

# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

C:\Users\SASAPU TARUN\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SASAPU TARUN\.cache\huggingface\hub\datasets--glue. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


...

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.613361,0.683824



KeyboardInterrupt



In [7]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6133608222007751, 'eval_accuracy': 0.6838235294117647}


In [8]:
model.save_pretrained("./ravdess_emotion_model")
feature_extractor.save_pretrained("./ravdess_emotion_feature_extractor")

NameError: name 'feature_extractor' is not defined